In [1]:
import pandas as pd

In [2]:
%matplotlib inline

## Read data and separate Switzerland, Cantons, Districts, Communes

### Load the Excel with all the population data in it

In [3]:
df = pd.read_excel('CleanData/Einb-Wide-1981-2015.xls')

In [4]:
df.head(2)

,Einheit,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Schweiz,14299,12723,11831,11850,14393,14416,12370,11356,10342,...,46711,43889,44365,43440,39314,36012,33500,34061,32836,40689
1,- Zürich,2562,2654,2104,2758,3229,2687,2187,2255,1670,...,11494,9008,11156,11551,10365,7756,8532,7907,7679,9625


In [5]:
#Make sure we don't have any NaN in there
df = df[df['Einheit'].notnull()]

### Load some helper files

In [6]:
#Load the numbers and abbreviations file for the cantons
df_kant_ids = pd.read_csv('OriginalData/Kantone-ABK-ID.csv')
df_kant_ids.head(2)

,Kanton_ABK,Kanton_ID,Kanton_Name,Kanton_Name2
0,ZH,1,Zürich,Zürich
1,BE,2,Bern,Bern / Berne


In [7]:
#Load the canton/district infos for the communes
df_gem_ids = pd.read_excel('OriginalData/Gemeindestand-2015.xls')
df_gem_ids.head(2)

,Hist.-Nummer,Kanton,Bezirksnummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,13256,ZH,101,Affoltern,1,Aeugst am Albis,1976-11-15
1,11742,ZH,101,Affoltern,2,Affoltern am Albis,1960-01-01


In [8]:
df_gem_ids.pop('Hist.-Nummer')
df_gem_ids.pop('Gemeindename')
df_gem_ids.pop('Datum der Aufnahme')
df_gem_ids.rename(columns={'Bezirksnummer': 'Bezirk_ID', 'Bezirksname': 'Bezirk_Name', 'BFS Gde-nummer': 'Gemeinde_ID', 'Kanton': 'Kanton_ABK'}, inplace=True)
df_gem_ids.head(2)

,Kanton_ABK,Bezirk_ID,Bezirk_Name,Gemeinde_ID
0,ZH,101,Affoltern,1
1,ZH,101,Affoltern,2


In [9]:
#Construct a set of district ids
df_bez_ids = df_gem_ids.drop_duplicates(subset='Bezirk_ID')
df_bez_ids.pop('Gemeinde_ID')
df_bez_ids.head(2)

,Kanton_ABK,Bezirk_ID,Bezirk_Name
0,ZH,101,Affoltern
14,ZH,102,Andelfingen


## Get a dataframe for each geopraphic level

### Separate Switzerland

In [10]:
#Switzerland
df_ch = df[df['Einheit'] == 'Schweiz']
df_ch.pop('Einheit')
df_ch.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,14299,12723,11831,11850,14393,14416,12370,11356,10342,8658,...,46711,43889,44365,43440,39314,36012,33500,34061,32836,40689


In [11]:
df_ch.shape

(1, 35)

### Separate the cantons

In [12]:
#Cantons - there should be 26 of them
df_kant = df[df['Einheit'].str.contains('^- ')]
df_kant['Kanton_Name'] = df_kant['Einheit'].str.extract('^- (.*)', expand=True)
df_kant.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Einheit,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Kanton_Name
1,- Zürich,2562,2654,2104,2758,3229,2687,2187,2255,1670,...,9008,11156,11551,10365,7756,8532,7907,7679,9625,Zürich
183,- Bern / Berne,846,817,930,810,824,1086,705,747,718,...,4301,4499,3030,2753,2886,2525,2545,1976,2619,Bern / Berne


In [13]:
#Add the ID and Abbreviations
df_kant = df_kant.merge(df_kant_ids, how='inner', left_on='Kanton_Name', right_on='Kanton_Name2')
df_kant.pop('Kanton_Name2')
df_kant.pop('Einheit')
df_kant.pop('Kanton_Name_y')
df_kant.rename(columns={'Kanton_Name_x': 'Kanton_Name'}, inplace=True)
df_kant.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2009,2010,2011,2012,2013,2014,2015,Kanton_Name,Kanton_ABK,Kanton_ID
0,2562,2654,2104,2758,3229,2687,2187,2255,1670,1598,...,11551,10365,7756,8532,7907,7679,9625,Zürich,ZH,1
1,846,817,930,810,824,1086,705,747,718,620,...,3030,2753,2886,2525,2545,1976,2619,Bern / Berne,BE,2


In [14]:
df_kant.shape

(26, 38)

### Separate the communes

In [15]:
#Gemeinden - there should be 2324 of them
df_gem = df[df['Einheit'].str.contains('^\.\.\.\.\.\.')]
df_gem['Gemeinde_ID'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\.(\d\d\d\d)', expand=True).astype(int)
df_gem['Gemeinde_Name'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\.\d\d\d\d (.*)', expand=True)
df_gem.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Einheit,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2008,2009,2010,2011,2012,2013,2014,2015,Gemeinde_ID,Gemeinde_Name
3,......0001 Aeugst am Albis,1,0,1,3,0,6,2,2,1,...,10,8,4,15,6,11,8,10,1,Aeugst am Albis
4,......0002 Affoltern am Albis,13,37,14,33,15,18,16,14,15,...,150,109,106,49,55,83,49,55,2,Affoltern am Albis


In [16]:
#Add the district and canton IDs and names
df_gem = df_gem.merge(df_gem_ids, how='inner', on='Gemeinde_ID')
df_gem.pop('Einheit')
df_gem.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2011,2012,2013,2014,2015,Gemeinde_ID,Gemeinde_Name,Kanton_ABK,Bezirk_ID,Bezirk_Name
0,1,0,1,3,0,6,2,2,1,0,...,15,6,11,8,10,1,Aeugst am Albis,ZH,101,Affoltern
1,13,37,14,33,15,18,16,14,15,12,...,49,55,83,49,55,2,Affoltern am Albis,ZH,101,Affoltern


In [17]:
df_gem = df_gem.merge(df_kant_ids, how='inner', on='Kanton_ABK')
df_gem.pop('Kanton_Name2')
df_gem.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2013,2014,2015,Gemeinde_ID,Gemeinde_Name,Kanton_ABK,Bezirk_ID,Bezirk_Name,Kanton_ID,Kanton_Name
0,1,0,1,3,0,6,2,2,1,0,...,11,8,10,1,Aeugst am Albis,ZH,101,Affoltern,1,Zürich
1,13,37,14,33,15,18,16,14,15,12,...,83,49,55,2,Affoltern am Albis,ZH,101,Affoltern,1,Zürich


In [18]:
df_gem.shape

(2324, 42)

### Separate the districts

In [19]:
#We have to do this one differently
df_bez = df_gem.groupby('Bezirk_ID').sum().reset_index()
df_bez.pop('Gemeinde_ID')
df_bez.pop('Kanton_ID')
df_bez.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,101,50,77,40,80,68,74,49,60,46,...,282,273,319,262,244,209,214,242,195,267
1,102,31,7,38,43,48,23,20,36,20,...,74,84,126,115,75,95,89,50,64,86


In [20]:
df_bez = df_bez.merge(df_bez_ids, how='inner', on='Bezirk_ID')
df_bez.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2008,2009,2010,2011,2012,2013,2014,2015,Kanton_ABK,Bezirk_Name
0,101,50,77,40,80,68,74,49,60,46,...,319,262,244,209,214,242,195,267,ZH,Affoltern
1,102,31,7,38,43,48,23,20,36,20,...,126,115,75,95,89,50,64,86,ZH,Andelfingen


In [21]:
df_bez = df_bez.merge(df_kant_ids, how='inner', on='Kanton_ABK')
df_bez.pop('Kanton_Name2')
df_bez.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2010,2011,2012,2013,2014,2015,Kanton_ABK,Bezirk_Name,Kanton_ID,Kanton_Name
0,101,50,77,40,80,68,74,49,60,46,...,244,209,214,242,195,267,ZH,Affoltern,1,Zürich
1,102,31,7,38,43,48,23,20,36,20,...,75,95,89,50,64,86,ZH,Andelfingen,1,Zürich


## Export the data

In [22]:
#Switzerland
df_ch.to_csv('CleanerData/CH-Einb-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/CH-Einb-Wide-1981-2015.csv')
df_temp.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,14299,12723,11831,11850,14393,14416,12370,11356,10342,8658,...,46711,43889,44365,43440,39314,36012,33500,34061,32836,40689


In [23]:
#Cantons
df_kant.to_csv('CleanerData/Kant-Einb-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/Kant-Einb-Wide-1981-2015.csv')
df_temp.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2009,2010,2011,2012,2013,2014,2015,Kanton_Name,Kanton_ABK,Kanton_ID
0,2562,2654,2104,2758,3229,2687,2187,2255,1670,1598,...,11551,10365,7756,8532,7907,7679,9625,Zürich,ZH,1
1,846,817,930,810,824,1086,705,747,718,620,...,3030,2753,2886,2525,2545,1976,2619,Bern / Berne,BE,2


In [24]:
#Districts
df_bez.to_csv('CleanerData/Bez-Einb-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/Bez-Einb-Wide-1981-2015.csv')
df_temp.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2010,2011,2012,2013,2014,2015,Kanton_ABK,Bezirk_Name,Kanton_ID,Kanton_Name
0,101,50,77,40,80,68,74,49,60,46,...,244,209,214,242,195,267,ZH,Affoltern,1,Zürich
1,102,31,7,38,43,48,23,20,36,20,...,75,95,89,50,64,86,ZH,Andelfingen,1,Zürich


In [25]:
#Communes
df_gem.to_csv('CleanerData/Gem-Einb-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/Gem-Einb-Wide-1981-2015.csv')
df_temp.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2013,2014,2015,Gemeinde_ID,Gemeinde_Name,Kanton_ABK,Bezirk_ID,Bezirk_Name,Kanton_ID,Kanton_Name
0,1,0,1,3,0,6,2,2,1,0,...,11,8,10,1,Aeugst am Albis,ZH,101,Affoltern,1,Zürich
1,13,37,14,33,15,18,16,14,15,12,...,83,49,55,2,Affoltern am Albis,ZH,101,Affoltern,1,Zürich
